# BarcodeSeqKit Core

> Core data structures for barcode extraction from sequencing data

## Introduction

BarcodeSeqKit is a library designed for extracting and processing barcoded sequences from next-generation sequencing data. This notebook contains the core data structures and base classes for the library.


In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

## Introduction

BarcodeSeqKit is a library designed for extracting and processing barcoded sequences from next-generation sequencing data. This notebook contains the core data structures and utility functions for the library.

In [ ]:
#| export
import os
import re
import sys
import json
import enum
import logging
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Set, Optional, Union, Iterator, Any
from Bio.Seq import Seq

## Enumerations
Let's define the basic enumerations used throughout the library.

In [ ]:
#| export
class OrientationType(enum.Enum):
    """Orientation types for barcode sequences."""
    FORWARD = "FR"
    REVERSE_COMPLEMENT = "RC"

In [ ]:
#| export
class BarcodeLocationType(enum.Enum):
    """Location types for barcodes."""
    FIVE_PRIME = "5"
    THREE_PRIME = "3"
    UNKNOWN = "UNK"

In [ ]:
#| export

@dataclass
class BarcodeConfig:
    """Configuration for a barcode sequence."""
    sequence: str
    location: BarcodeLocationType = BarcodeLocationType.UNKNOWN
    name: Optional[str] = None
    description: Optional[str] = None
    
    def __post_init__(self):
        # Clean the sequence and ensure uppercase
        self.sequence = self.sequence.strip().upper()
        # If no name is provided, use the sequence as the name
        if self.name is None:
            self.name = self.sequence
    
    @property
    def reverse_complement(self) -> str:
        """Return the reverse complement of the barcode sequence."""
        return str(Seq(self.sequence).reverse_complement())
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert the barcode configuration to a dictionary."""
        return {
            "sequence": self.sequence,
            "location": self.location.value,
            "name": self.name,
            "description": self.description,
            "reverse_complement": self.reverse_complement
        }



@dataclass
class BarcodeExtractorConfig:
    """Configuration for barcode extraction."""
    barcodes: List[BarcodeConfig]
    output_prefix: str
    output_dir: str = "."
    max_mismatches: int = 0
    search_softclipped: bool = False
    verbose: bool = False
    log_file: Optional[str] = None
    write_output_files: bool = True  # New parameter to control output file generation
    
    def __post_init__(self):
        # Create output directory if it doesn't exist and we're writing files
        if self.write_output_files:
            os.makedirs(self.output_dir, exist_ok=True)
        
        # Set up logging
        self._setup_logging()
    
    def _setup_logging(self) -> None:
        """Set up logging configuration."""
        self.logger = logging.getLogger("BarcodeSeqKit")
        self.logger.setLevel(logging.DEBUG if self.verbose else logging.INFO)
        
        # Clear existing handlers
        self.logger.handlers = []
        
        # Console handler
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        console_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        console_handler.setFormatter(console_formatter)
        self.logger.addHandler(console_handler)
        
        # File handler (if requested)
        if self.log_file:
            file_handler = logging.FileHandler(self.log_file)
            file_handler.setLevel(logging.DEBUG)
            file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            file_handler.setFormatter(file_formatter)
            self.logger.addHandler(file_handler)
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert the configuration to a dictionary."""
        return {
            "barcodes": [barcode.to_dict() for barcode in self.barcodes],
            "output_prefix": self.output_prefix,
            "output_dir": self.output_dir,
            "max_mismatches": self.max_mismatches,
            "search_softclipped": self.search_softclipped,
            "verbose": self.verbose,
            "log_file": self.log_file,
            "write_output_files": self.write_output_files
        }
    
    def save_yaml(self, output_path: str) -> None:
        """Save the configuration to a YAML file."""
        import yaml
        with open(output_path, 'w') as f:
            yaml.dump(self.to_dict(), f, default_flow_style=False)
    
    @classmethod
    def load_yaml(cls, input_path: str) -> 'BarcodeExtractorConfig':
        """Load configuration from a YAML file."""
        import yaml
        with open(input_path, 'r') as f:
            config_dict = yaml.safe_load(f)
        
        # Extract barcode configurations
        barcode_configs = []
        for barcode_dict in config_dict.get("barcodes", []):
            location = BarcodeLocationType(barcode_dict.get("location", "UNK"))
            barcode_configs.append(BarcodeConfig(
                sequence=barcode_dict["sequence"],
                location=location,
                name=barcode_dict.get("name"),
                description=barcode_dict.get("description")
            ))
        
        # Create and return the configuration
        return cls(
            barcodes=barcode_configs,
            output_prefix=config_dict["output_prefix"],
            output_dir=config_dict.get("output_dir", "."),
            max_mismatches=config_dict.get("max_mismatches", 0),
            search_softclipped=config_dict.get("search_softclipped", False),
            verbose=config_dict.get("verbose", False),
            log_file=config_dict.get("log_file"),
            write_output_files=config_dict.get("write_output_files", True)
        )  

In [ ]:
#| export
@dataclass
class BarcodeMatch:
    """Represents a match of a barcode in a sequence."""
    barcode: BarcodeConfig
    orientation: OrientationType
    position: int
    sequence: str
    
    def __str__(self) -> str:
        """String representation of the barcode match."""
        return f"{self.barcode.name} ({self.orientation.value}) at position {self.position}"


## Statistics Tracking

In [ ]:
#| export
@dataclass
class ExtractionStatistics:
    """Statistics collected during barcode extraction."""
    total_reads: int = 0
    total_barcode_matches: int = 0
    matches_by_barcode: Dict[str, int] = field(default_factory=dict)
    matches_by_orientation: Dict[str, int] = field(default_factory=dict)
    matches_by_category: Dict[str, int] = field(default_factory=dict)
    
    def update_barcode_match(self, barcode_match: BarcodeMatch, category: Optional[str] = None) -> None:
        """Update statistics based on a barcode match.
        
        Args:
            barcode_match: The barcode match
            category: The category (e.g., 'barcode5_orientFR'). If None, will be derived from the match.
        """
        self.total_barcode_matches += 1
        
        # Update matches by barcode
        barcode_name = barcode_match.barcode.name
        if barcode_name not in self.matches_by_barcode:
            self.matches_by_barcode[barcode_name] = 0
        self.matches_by_barcode[barcode_name] += 1
        
        # Update matches by orientation
        orientation = barcode_match.orientation.value
        if orientation not in self.matches_by_orientation:
            self.matches_by_orientation[orientation] = 0
        self.matches_by_orientation[orientation] += 1
        
        # Derive category if not provided
        if category is None:
            # Try to derive a reasonable category
            location = barcode_match.barcode.location.value
            if location in ["5", "3"]:
                category = f"barcode{location}_orient{orientation}"
            else:
                category = f"barcode_orient{orientation}"
        
        # Update matches by category
        if category not in self.matches_by_category:
            self.matches_by_category[category] = 0
        self.matches_by_category[category] += 1
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert the statistics to a dictionary."""
        return {
            "total_reads": self.total_reads,
            "total_barcode_matches": self.total_barcode_matches,
            "matches_by_barcode": self.matches_by_barcode,
            "matches_by_orientation": self.matches_by_orientation,
            "matches_by_category": self.matches_by_category,
            "match_rate": (self.total_barcode_matches / self.total_reads) if self.total_reads > 0 else 0
        }
    
    def save_json(self, output_path: str) -> None:
        """Save the statistics to a JSON file."""
        with open(output_path, 'w') as f:
            json.dump(self.to_dict(), f, indent=2)
    
    def save_tsv(self, output_path: str) -> None:
        """Save the statistics to a TSV file."""
        with open(output_path, 'w') as f:
            # Write header
            f.write("Metric\tValue\n")
            
            # Write overall statistics
            stats_dict = self.to_dict()
            f.write(f"TotalReads\t{stats_dict['total_reads']}\n")
            f.write(f"TotalBarcodeMatches\t{stats_dict['total_barcode_matches']}\n")
            f.write(f"MatchRate\t{stats_dict['match_rate']:.4f}\n")
            
            # Write barcode-specific statistics
            f.write("\nBarcode\tCount\n")
            for barcode, count in stats_dict['matches_by_barcode'].items():
                f.write(f"{barcode}\t{count}\n")
            
            # Write orientation-specific statistics
            f.write("\nOrientation\tCount\n")
            for orientation, count in stats_dict['matches_by_orientation'].items():
                f.write(f"{orientation}\t{count}\n")
            
            # Write category-specific statistics
            f.write("\nCategory\tCount\n")
            for category, count in stats_dict['matches_by_category'].items():
                f.write(f"{category}\t{count}\n")

## File Format Utilities

In [ ]:
#| export
class FileFormat(enum.Enum):
    """Supported file formats."""
    BAM = "BAM"
    FASTQ = "FASTQ"
    UNKNOWN = "UNKNOWN"

    @classmethod
    def detect_format(cls, file_path: str) -> 'FileFormat':
        """Detect the format of a file based on its extension."""
        lower_path = file_path.lower()
        if lower_path.endswith('.bam'):
            return cls.BAM
        elif any(lower_path.endswith(ext) for ext in ['.fastq', '.fq', '.fastq.gz', '.fq.gz']):
            return cls.FASTQ
        elif os.path.isdir(file_path):
            # Check if directory contains FASTQ files
            for f in os.listdir(file_path):
                if any(f.lower().endswith(ext) for ext in ['.fastq', '.fq', '.fastq.gz', '.fq.gz']):
                    return cls.FASTQ
            return cls.UNKNOWN
        else:
            return cls.UNKNOWN

## Example Usage


In [ ]:
# Example of creating barcode configurations
barcode_5prime = BarcodeConfig(
    sequence="TCGCGAGGC",
    location=BarcodeLocationType.FIVE_PRIME,
    name="5prime",
    description="5' barcode for phenotyping experiment"
)

barcode_3prime = BarcodeConfig(
    sequence="GGCCGGCCGG",
    location=BarcodeLocationType.THREE_PRIME,
    name="3prime",
    description="3' barcode for phenotyping experiment"
)
# Print information about the barcodes
print(f"5' barcode: {barcode_5prime.sequence}, RC: {barcode_5prime.reverse_complement}")
print(f"3' barcode: {barcode_3prime.sequence}, RC: {barcode_3prime.reverse_complement}")

# Create a basic extraction configuration
config = BarcodeExtractorConfig(
    barcodes=[barcode_5prime, barcode_3prime],
    output_prefix="test_output",
    output_dir="../tests/core_output",
    max_mismatches=0,
    verbose=True
)

# Save the configuration to YAML
config.save_yaml("../tests/core_output/test_config.yaml")
print("Configuration saved to test_config.yaml")

# Load the configuration back
loaded_config = BarcodeExtractorConfig.load_yaml("../tests/core_output/test_config.yaml")
print(f"Loaded {len(loaded_config.barcodes)} barcodes from config file")

5' barcode: TCGCGAGGC, RC: GCCTCGCGA
3' barcode: GGCCGGCCGG, RC: CCGGCCGGCC
Configuration saved to test_config.yaml
Loaded 2 barcodes from config file


## Basic Utility Functions
Let's add some basic utility functions for common operations.

## Conclusion
This notebook establishes the core data structures for the BarcodeSeqKit library It provides all the necessary components for barcode configuration, matching, and statistics tracking, which will be used by the specialized processors for BAM and FASTQ files.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()